In [1]:
import random as rd
import numpy as np

In [2]:
def dropColumns(A, threshold): #this has problems when large numbers are concerned
    t = 0
    l = []
    tmp = np.matrix([[0],[0]])
    for t in range(A.shape[1]):
        if A[1, t] > threshold:
            l.append(t)
        t += 1
    for n in l: 
        c = np.matrix(A[:, n])
        c = np.transpose(c)
        tmp = np.hstack((tmp,c))
    
    A = tmp[:, 1:]
    return A

def dropCol2 (A, threshold):
    l = []
    for i in range(A.shape[1]):
        tmp = shapeTrans(A[:, i], 1)
        tmp = tmp[0]
        l.append(tmp)
    
    toKeep = [col for col in l if col[1] > threshold]
    A = np.transpose(np.matrix(toKeep))
    return A

def shapeTrans(A, n):
    'transforms a matrix of shape (2,n) into a lists of n 2-dim lists'

    l = []
    k  = 0
    while k < n: 
        l.append([A[0, k], A[1, k]])
        k += 1
    return l

def shapeTransInv(l, n):
    'transforms a lists of n 2-dim lists into a matrix of shape (2,n)'
    copy = []
    for i in range(len(l)):
        copy.append([[l[i][0]],[l[i][1]]])
        i += 1
    A = np.matrix([[0],[0]])
    k = 0
    while k <n:
        A = np.hstack((A,copy[k]))
        k += 1
    A = A [:, 1:]
    return A

def getKey(item):
    return item[1]




class Person:
    def __init__(self, number, partnersMemory, memoryCapacity, fitness, turnPayoff, alwaysDefect, pastWeighting, thresholdCoop, cliquishness, pairingError, \
                     strategyError, doCooperate, didOtherCoop, isPreferredPartner, amFree, myPartner, Pdeath, Prepr):
        self.number = number
        self.partnersMemory = partnersMemory #matrice 2 x K
        self.memoryCapacity = memoryCapacity #initially must be an empty matrix! not even with zeros! First row names, second prefs
        self.fitness = fitness
        self.turnPayoff = turnPayoff
        #self.payoffMatrix = payoff.Matrix
        self.alwaysDefect = alwaysDefect
        self.pastWeighting = pastWeighting
        self.thresholdCoop = thresholdCoop
        self.cliquishness = cliquishness
        self.pairingError = pairingError
        self.strategyError = strategyError
        self.doCooperate = doCooperate
        self.didOtherCoop = didOtherCoop
        self.isPreferredPartner = isPreferredPartner
        #self.isMemoryFull = isMemoryFull
        self.amFree = amFree
        self.myPartner = myPartner
        self.Pdeath = Pdeath
        self.Prepr = Prepr
        #self.myTime = myTime
        #self.worldXSize = worldXSize
        #self.worldYSize = worldYSize
    
    def interact(self): #NB: at step WO! I'm for the moment cutting out the possibility to not coop even if memory not full part from strategy err
            p = rd.uniform(0,1)
            if self.alwaysDefect == 1:
                if p < 1 - self.strategyError:
                    self.doCooperate = False
                else:
                    self.doCooperate = True
            else:
                if self.isPreferredPartner == True:
                    if p < 1 - self.strategyError:
                        self.doCooperate = True
                    else:
                        self.doCooperate = False
                else:
                    if self.partnersMemory.shape[1] == self.memoryCapacity:
                        if p < self.cliquishness:
                            if p < 1 - self.strategyError:
                                self.doCooperate = False
                            else: 
                                self.doCooperate = True
                        else: #WO! 
                            if p < 1 - self.strategyError:
                                self.doCooperate = True
                            else:
                                self.doCooperate = False
                    else:
                        if p < 1 - strategyError:
                            self.doCooperate = True
                        else:
                            self.doCooperate = False
                            
    def update(self, generalPayoffMatrix, alpha=0.6):
        'updates the parameters of the agent, if not changed alpha is set to 0.6 as in the paper'
        #update fitness
        self.fitness = self.turnPayoff + alpha * self.fitness

        #update partners memory
        #when the matrix is empty it adjusts the input values
        if self.partnersMemory.shape == (2,0):
            turnPref = (1 - self.pastWeighting) * self.turnPayoff / generalPayoffMatrix[1, 0]
            l = ([[self.myPartner],[turnPref]])
            self.partnersMemory = np.matrix(l)
        else:
            #does the agent have the partner in memory?
            j = 0
            isItThere = False
            while j < self.partnersMemory.shape[1]:
                if self.myPartner != self.partnersMemory[0, j]:
                    isItThere = False
                    j += 1
                else: 
                    isItThere = True
                    myFriendIndex = j 
                    j = self.partnersMemory.shape[1]
            
            #If it doesn't and if the other agent cooperated        
            if isItThere == False and self.didOtherCoop == True:
                turnPref = (1 - self.pastWeighting) * self.turnPayoff / generalPayoffMatrix[1, 0]
                if self.partnersMemory.shape[1] < self.memoryCapacity:
                    l = [[self.myPartner],[turnPref]]
                    self.partnersMemory = np.hstack((self.partnersMemory,l))
                else:
                    k = 0
                    betterExp = False
                    while k in range(self.partnersMemory.shape[1]): 
                        if turnPref <= self.partnersMemory[1, k]:
                            k += 1
                        else: 
                            betterExp = True
                            k = self.partnersMemory.shape[1]
                    if betterExp == True:
                        self.partnersMemory[0, self.memoryCapacity-1] = self.myPartner
                        self.partnersMemory[1, self.memoryCapacity-1] = turnPref
            #if it does, whatever the partner does, the agent will remember of him
            elif isItThere == True:
                turnPref = self.pastWeighting * self.partnersMemory[1, myFriendIndex] + (1 - self.pastWeighting) * self.turnPayoff / generalPayoffMatrix[1, 0]
                self.partnersMemory[1, myFriendIndex] = turnPref

        #remove uncoop partners
        #if self.partnersMemory.shape[1] == self.memoryCapacity:
        dropCol2(self.partnersMemory, self.thresholdCoop)

        #orders memory
        mem = shapeTrans(self.partnersMemory, self.partnersMemory.shape[1])
        mem = sorted(mem, key = getKey, reverse=True)
        self.partnersMemory = shapeTransInv(mem, len(mem))
        
        #be free again!
        self.amFree = True
    
    def choosePartner(self, listOfAgents):
        if self.partnersMemory.shape == (2,0):
            index = False
            while index == False:
                desideredPartner = rd.choice(listOfAgents)
                if desideredPartner.amFree == True and desideredPartner.number != self.number:
                    self.myPartner = desideredPartner.number
                    desideredPartner.myPartner = self.number
                    self.amFree = False
                    desideredPartner.amFree = False
                    #print("Now my partner is agent ", desideredPartner.number)
                    index = True
                else: 
                    index = False
        else:
            p = rd.uniform (0,1)
            i = 0 
            while i in range(self.partnersMemory.shape[1]) and self.amFree == True:
                for person in listOfAgents:
                    #print("sono al ciclo di agenti!")
                    bcontrol = False
                    if person.number == self.partnersMemory[0, i] and person.number != self.number: #I am supposing of having an ordered matrix by each agent's score 
                        ind = 0
                        while ind in range(person.partnersMemory.shape[1])and bcontrol == False:
                            if person.partnersMemory[0, ind] != self.number:
                                bcontrol = False
                            else:
                                bcontrol = True
                            ind += 1
                        
                    if person.amFree == True and bcontrol == True and p < 1 - self.pairingError:
                        self.myPartner = person.number
                        person.myPartner = self.number
                        self.amFree = False
                        person.amFree = False
                        desideredPartner = person
                    #else: 
                        #print("non è il mio amichetto, passo a quello dopo")
                i += 1
                
            if self.amFree == True:
                #print("Sono ancora libero")
                listOfUnpaired = []
                for person in listOfAgents:
                    if person.amFree == True and person.number != self.number:
                        listOfUnpaired.append(person)
                #print("Creata lista di agenti spaiati")    
                desideredPartner = rd.sample(listOfUnpaired, 1)[0]
                self.myPartner = desideredPartner.number
                desideredPartner.myPartner = self.number
                self.amFree = False
                desideredPartner.amFree = False                
        return desideredPartner

